# Clockify User Sync & Identity Automation

This notebook provides a Google Colab-compatible interface for synchronizing user profiles, custom fields, and manager hierarchies from a CSV source-of-truth into a Clockify workspace.

## Setup
1. Upload your `config.yaml` and CSV file using the file browser (left sidebar)
2. Run each cell in order
3. Enter your API key and workspace ID when prompted

In [ ]:
# Install dependencies
!pip install pandas requests pyyaml typer

In [ ]:
import yaml
import pandas as pd
import requests
import time
import getpass
from typing import Dict, List, Optional, Any, Set

In [ ]:
# --- Configuration ---
# Upload your config.yaml or edit these values directly

CONFIG = {
    "workspace": {
        "fallback_manager_email": "example@example.com",
        "fallback_group_name": "Other functions"
    },
    "field_mapping": {
        "Company (Label)": "Company",
        "Country (Label)": "Country",
        "Global Id": "Global ID"
    },
    "api": {
        "base_url": "https://api.clockify.me/api/v1",
        "rate_limit_delay": 0.15,
        "max_retries": 3
    }
}

# Or load from file:
# with open('config.yaml', 'r') as f:
#     CONFIG = yaml.safe_load(f)

In [ ]:
# --- Credentials ---
API_KEY = getpass.getpass("Enter Clockify API Key: ")
WORKSPACE_ID = input("Enter Workspace ID: ")

In [ ]:
class ClockifySyncEngine:
    def __init__(self, config: dict, api_key: str, workspace_id: str, dry_run: bool = False):
        self.config = config
        self.api_key = api_key
        self.workspace_id = workspace_id
        self.dry_run = dry_run
        self.base_url = self.config['api']['base_url']
        self.headers = {
            "X-Api-Key": self.api_key,
            "content-type": "application/json"
        }
        self.fallback_manager_email = self.config['workspace']['fallback_manager_email']
        self.fallback_group_name = self.config['workspace']['fallback_group_name']
        self.field_mapping = self.config['field_mapping']
        self.success_log = []
        self.error_log = []

    def log_api(self, email, action, details, response=None, error_msg=None):
        status_code = response.status_code if response is not None else 0
        if response is None and not error_msg:
            if action == "Fallback Assignment": status_code = 204
            else: status_code = 599; error_msg = "No Response"
        response_text = response.text if response is not None else (error_msg or "No Response")
        response_text_lower = response_text.lower() if response_text else ""
        is_cleanup_success = (action.startswith("Wipe") or action.startswith("Remove") or action.startswith("Deactivate")) and status_code in [400, 404]
        idempotent_actions = {"Add to Group", "Assign Direct Mgr", "Assign Group Mgr", "Create Group", "Create Custom Field", "Auto-Fix Group Mgr", "Pre-Deactivation Reassign"}
        is_idempotent_conflict = action in idempotent_actions and status_code in [400, 409] and any(
            k in response_text_lower for k in ["already", "exists", "duplicate"]
        )
        is_info_log = status_code == 0 and not error_msg
        if (200 <= status_code < 300) or is_cleanup_success or is_idempotent_conflict:
            icon = "\u2705"
            if is_cleanup_success: details += " (Already clean)"
            if is_idempotent_conflict: details += " (Already set)"
        elif is_info_log:
            icon = "\u2139\ufe0f"
        else:
            icon = "\u274c"
        print(f"{icon} [{status_code}] {email}: {action} - {details}")
        if status_code >= 400 and not is_cleanup_success and not is_idempotent_conflict:
            print(f"      \u21b3 Msg: {response_text[:200]}...")
        entry = {"Email": email, "Action": action, "Details": details, "Status": status_code, "Response": response_text}
        if (200 <= status_code < 300) or is_cleanup_success or is_idempotent_conflict or is_info_log:
            self.success_log.append(entry)
        else:
            self.error_log.append(entry)

    def request(self, method, endpoint, json=None):
        if self.dry_run and method in ["POST", "PATCH", "PUT", "DELETE"]:
            res = requests.Response()
            res.status_code = 200
            res._content = b'{"dry_run": true}'
            return res
        time.sleep(self.config['api'].get('rate_limit_delay', 0.15))
        url = f"{self.base_url}{endpoint}"
        for retry in range(self.config['api'].get('max_retries', 3)):
            try:
                res = requests.request(method, url, headers=self.headers, json=json)
                if res.status_code == 429:
                    wait = (2 ** retry)
                    print(f"      \u26a0\ufe0f Rate limited. Waiting {wait}s...")
                    time.sleep(wait)
                    continue
                return res
            except Exception as e:
                print(f"      \u274c Connection error: {e}")
                if retry == self.config['api'].get('max_retries', 3) - 1: return None
        return None

    def get_all_items(self, endpoint):
        items = []
        page = 1
        page_size = 1000
        while True:
            separator = "&" if "?" in endpoint else "?"
            url = f"{endpoint}{separator}page={page}&page-size={page_size}"
            if "users" in endpoint and "user-groups" not in endpoint:
                url += "&memberships=WORKSPACE&include-roles=true"
            res = self.request("GET", url)
            if not res or res.status_code != 200: break
            data = res.json()
            if not data: break
            items.extend(data)
            if len(data) < page_size: break
            page += 1
        return items

    def preflight_check(self, groups, protected_ids):
        probe_group = None
        for g in groups:
            if g['id'] not in protected_ids:
                probe_group = g
                break
        if not probe_group: return True
        print(f"\U0001f50d Preflight: Probing permissions using group '{probe_group['name']}'...")
        user_id = probe_group.get('userIds', [None])[0]
        if not user_id: return True
        res = self.request("POST", f"/workspaces/{self.workspace_id}/user-groups/{probe_group['id']}/users", {"userId": user_id})
        if res and res.status_code in [401, 403]:
            print(f"\u274c Preflight failed: {res.text}")
            return False
        return True

    def ensure_custom_fields(self, cfs):
        cf_map = {}
        required = list(self.field_mapping.values())
        for cf in cfs:
            if cf.get('entityType') == 'USER':
                cf_map[cf['name']] = cf['id']
        for name in required:
            if name not in cf_map:
                print(f"\u2795 Creating Custom Field: {name}...")
                res = self.request("POST", f"/workspaces/{self.workspace_id}/custom-fields",
                                   {"name": name, "type": "TXT", "entityType": "USER", "status": "VISIBLE"})
                if res and res.status_code in [200, 201]:
                    cf_map[name] = res.json()['id']
        return cf_map

    def get_display_name(self, user):
        return (user.get('name') or user.get('email') or '').strip()

    def is_active(self, user):
        for m in user.get('memberships', []):
            if m.get('targetId') == self.workspace_id: return m.get('membershipStatus') == 'ACTIVE'
        return user.get('status') == 'ACTIVE'

    def clean_number(self, val):
        try: return float(str(val).replace(',', '.'))
        except: return 0.0

    def to_iso8601(self, hrs):
        h = int(hrs); m = int(round((hrs - h) * 60))
        return f"PT{h}H{m}M"

    def save_logs(self):
        pd.DataFrame(self.success_log).to_csv("sync_success_log.csv", index=False)
        pd.DataFrame(self.error_log).to_csv("sync_error_log.csv", index=False)
        print(f"\nLogs saved: {len(self.success_log)} successes, {len(self.error_log)} errors")

    def run(self, csv_path, cleanup=False, deactivate=False):
        df = pd.read_csv(csv_path)
        df.columns = df.columns.str.strip()
        print("\n\u23f3 Fetching workspace data...")
        users_raw = self.get_all_items(f"/workspaces/{self.workspace_id}/users")
        user_map = {u['email'].strip().lower(): u for u in users_raw if u.get('email')}
        groups_raw = self.get_all_items(f"/workspaces/{self.workspace_id}/user-groups")
        group_map = {g['name'].strip(): g for g in groups_raw}
        group_id_map = {g['id']: g for g in groups_raw}
        protected_names = set()
        if 'Country (Label)' in df.columns:
            protected_names = set(df['Country (Label)'].dropna().astype(str).str.strip().str.lower())
        protected_ids = {g['id'] for name, g in group_map.items() if name.lower() in protected_names}
        if not self.preflight_check(groups_raw, protected_ids): return
        cfs_raw = self.get_all_items(f"/workspaces/{self.workspace_id}/custom-fields?entity-type=USER")
        cf_id_map = self.ensure_custom_fields(cfs_raw)
        managed_group_names = {self.fallback_group_name}
        csv_manager_emails = set(df['Manager NTID email'].dropna().astype(str).str.strip().str.lower())
        for email in csv_manager_emails:
            if email in user_map:
                managed_group_names.add(self.get_display_name(user_map[email]))
        managed_ids = {g['id'] for name, g in group_map.items() if name in managed_group_names and g['id'] not in protected_ids}
        print(f"\n\u2622\ufe0f  PHASE 1: CLEANUP (Managed Groups: {len(managed_ids)})")
        for user in users_raw:
            roles = user.get('roles', [])
            for role in roles:
                if role.get('role') == 'TEAM_MANAGER':
                    for entity in role.get('entities', []):
                        if entity.get('id') in managed_ids or not entity.get('id'):
                            res = self.request("DELETE", f"/workspaces/{self.workspace_id}/users/{user['id']}/roles",
                                             {"entityId": entity.get('id'), "role": "TEAM_MANAGER"})
                            self.log_api(user['email'], "Wipe Role", f"Unassigned from {entity.get('id') or 'Ghost'}", res)
        for gid in managed_ids:
            g = group_id_map[gid]
            for uid in g.get('userIds', []):
                res = self.request("DELETE", f"/workspaces/{self.workspace_id}/user-groups/{gid}/users/{uid}")
        print("\n\U0001f680 PHASE 2: RECONSTRUCTION")
        for _, row in df.iterrows():
            email = str(row.get('NTID email', '')).strip().lower()
            if email not in user_map: continue
            user_id = user_map[email]['id']
            weekly_hrs = self.clean_number(row.get('Weekly Working Hours', 40))
            capacity = self.to_iso8601(weekly_hrs / 5)
            cfs_payload = []
            for csv_h, cf_n in self.field_mapping.items():
                if csv_h in row and cf_n in cf_id_map:
                    val = str(row[csv_h]) if not pd.isna(row[csv_h]) else ""
                    cfs_payload.append({"customFieldId": cf_id_map[cf_n], "value": val})
            res = self.request("PATCH", f"/workspaces/{self.workspace_id}/member-profile/{user_id}",
                             {"workCapacity": capacity, "userCustomFields": cfs_payload})
            self.log_api(email, "Update Profile", "Capacity/CFs", res)
            mgr_email = str(row.get('Manager NTID email', '')).strip().lower()
            target_mgr_id = None
            target_g_name = self.fallback_group_name
            if mgr_email in user_map and self.is_active(user_map[mgr_email]):
                target_mgr_id = user_map[mgr_email]['id']
                target_g_name = self.get_display_name(user_map[mgr_email])
            else:
                if mgr_email: print(f"      \u26a0\ufe0f Manager '{mgr_email}' invalid/inactive. Falling back.")
                if self.fallback_manager_email.lower() in user_map:
                    target_mgr_id = user_map[self.fallback_manager_email.lower()]['id']
            if target_g_name not in group_map and not self.dry_run:
                res = self.request("POST", f"/workspaces/{self.workspace_id}/user-groups", {"name": target_g_name})
                if res and res.status_code == 201:
                    group_map[target_g_name] = res.json()
                    group_id_map[res.json()['id']] = res.json()
            if target_g_name in group_map:
                gid = group_map[target_g_name]['id']
                self.request("POST", f"/workspaces/{self.workspace_id}/user-groups/{gid}/users", {"userId": user_id})
                self.request("POST", f"/workspaces/{self.workspace_id}/users/{target_mgr_id}/roles", {"entityId": user_id, "role": "TEAM_MANAGER"})
                self.request("POST", f"/workspaces/{self.workspace_id}/users/{target_mgr_id}/roles", {"entityId": gid, "role": "TEAM_MANAGER", "sourceType": "USER_GROUP"})
        print("\n\u2705 Sync complete.")
        self.save_logs()

In [ ]:
# --- Upload CSV and Run ---
# Option A: Upload via Colab sidebar, then set the path:
CSV_PATH = "users.csv"  # <-- change to your uploaded filename

# Option B: Upload interactively (Colab only)
# from google.colab import files
# uploaded = files.upload()
# CSV_PATH = list(uploaded.keys())[0]

In [ ]:
# --- Execute Sync ---
# Set dry_run=True to preview changes without modifying workspace

engine = ClockifySyncEngine(
    config=CONFIG,
    api_key=API_KEY,
    workspace_id=WORKSPACE_ID,
    dry_run=True  # Change to False for live execution
)

engine.run(csv_path=CSV_PATH)

In [ ]:
# --- Download Logs (Colab only) ---
# from google.colab import files
# files.download('sync_success_log.csv')
# files.download('sync_error_log.csv')